# 05 - modelo solución (LightGBM optimizado)
Preprocesado con OOF LightGBM con early stopping.

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json || true
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -p ./data || true
!unzip -o ./data/udea*.zip -d ./data > /dev/null 2>&1 || true
!ls -la ./data || true

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.30GB/s]
total 233348
drwxr-xr-x 2 root root      4096 Nov 26 03:16 .
drwxr-xr-x 1 root root      4096 Nov 26 03:16 ..
-rw-r--r-- 1 root root   4716673 Sep 16 01:46 submission_example.csv
-rw-r--r-- 1 root root  59185238 Sep 16 01:46 test.csv
-rw-r--r-- 1 root root 143732437 Sep 16 01:46 train.csv
-rw-r--r-- 1 root root  31301114 Sep 16 01:46 udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
print("libs listas")


libs listas


In [ ]:

train = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')
print("train", train.shape)
print("test ", test.shape)
train.head()

train (692500, 21)
test  (296786, 20)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [ ]:
TARGET = 'RENDIMIENTO_GLOBAL'
le = LabelEncoder()

# codificar target (importante para LGBM)
y = le.fit_transform(train[TARGET].astype(str).fillna('missing'))

X = train.drop(columns=[TARGET]).copy()
X_test = test.copy()

num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()

print('num_cols:', len(num_cols), 'cat_cols:', len(cat_cols))
print('ejemplos cat:', cat_cols[:10])

num_cols: 6 cat_cols: 14
ejemplos cat: ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD']


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

num_tr = Pipeline([('imputer', SimpleImputer(strategy='median'))])
cat_tr = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preproc = ColumnTransformer([
    ('num', num_tr, num_cols),
    ('cat', cat_tr, cat_cols)
], remainder='drop')

# Ajustar con train+test para evitar mismatch de categorías
X_all = pd.concat([X[num_cols+cat_cols], X_test[num_cols+cat_cols]], axis=0, ignore_index=True)
preproc.fit(X_all)
print("Preprocessor ajustado con train+test")

X_proc = preproc.transform(X[num_cols+cat_cols])
X_test_proc = preproc.transform(X_test[num_cols+cat_cols])
print("X_proc shape:", X_proc.shape, "X_test_proc shape:", X_test_proc.shape)

Preprocessor ajustado con train+test
X_proc shape: (692500, 20) X_test_proc shape: (296786, 20)


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np

def get_oof_lgb(params, X, y, X_test, n_splits=7, random_state=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    n_classes = len(np.unique(y))
    oof_train = np.zeros((X.shape[0], n_classes))
    oof_test = np.zeros((X_test.shape[0], n_classes))
    for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"Fold {fold+1}/{n_splits}")
        X_tr, X_val = X[tr_idx], X[val_idx]
        y_tr, y_val = y[tr_idx], y[val_idx]
        clf = LGBMClassifier(**params)
        clf.fit(X_tr, y_tr,
                eval_set=[(X_val, y_val)],
                eval_metric='multi_logloss',
                early_stopping_rounds=100,
                verbose=False)
        oof_train[val_idx, :] = clf.predict_proba(X_val)
        oof_test += clf.predict_proba(X_test) / n_splits
    return oof_train, oof_test

In [ ]:
params = dict(
    objective='multiclass',
    num_class=len(le.classes_),
    n_estimators=2500,
    learning_rate=0.02,
    num_leaves=128,
    max_depth=-1,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    min_child_samples=20,
    random_state=42,
    n_jobs=-1
)
print(params)

{'objective': 'multiclass', 'num_class': 4, 'n_estimators': 2500, 'learning_rate': 0.02, 'num_leaves': 128, 'max_depth': -1, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'min_child_samples': 20, 'random_state': 42, 'n_jobs': -1}


In [ ]:
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

def get_oof_lgb(params, X, y, X_test, n_splits=4, random_state=42):
    oof_preds = np.zeros((len(X), params["num_class"]))
    test_preds = np.zeros((len(X_test), params["num_class"]))

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y), 1):
        print(f"Fold {fold}/{n_splits}")

        X_tr, X_val = X[tr_idx], X[val_idx]
        y_tr, y_val = y[tr_idx], y[val_idx]

        clf = LGBMClassifier(**params)

        clf.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric="multi_logloss",
            callbacks=[
                early_stopping(50),
                log_evaluation(0)
            ],
        )

        oof_preds[val_idx] = clf.predict_proba(X_val)
        test_preds += clf.predict_proba(X_test) / n_splits

    return oof_preds, test_preds


In [ ]:
X = X_proc
X_test = X_test_proc

final_clf = LGBMClassifier(**params)
final_clf.fit(X, y)
final_proba = final_clf.predict_proba(X_test)
final_preds = final_proba.argmax(axis=1)
final_labels = le.inverse_transform(final_preds)

submission = pd.DataFrame({'ID': test['ID'], 'RENDIMIENTO_GLOBAL': final_labels})
submission_path = './data/submission_99_lgbm_final.csv'
submission.to_csv(submission_path, index=False)
print("Submission guardada en", submission_path)
submission.head()

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.144601 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1495
[LightGBM] [Info] Number of data points in

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Submission guardada en ./data/submission_99_lgbm_final.csv


,ID,RENDIMIENTO_GLOBAL
0,550236,medio-bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo


In [1]:
import joblib
joblib.dump(final_clf, './data/lgbm_final_model.joblib')


NameError: name 'final_clf' is not defined

In [ ]:
print("Comando para enviar a Kaggle:")
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f data/submission_99_lgbm_final.csv -m "99 - modelo solucion: LGBM optimizado"

Comando para enviar a Kaggle:
100% 4.06M/4.06M [00:00<00:00, 9.76MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia

Para enviar a Kaggle ejecuta:
100% 4.13M/4.13M [00:00<00:00, 5.67MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia